In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
from numpy import int0, array, asarray, zeros, reshape, double, arange, \
                  ma, log10, diff, mean, flipud, floor, pi, sqrt, size, fliplr, meshgrid, exp, sin, cos, radians, ceil, round

In [32]:
!rm -rf /tmp/*
!find /home/mag/Documents/repos/solab/PySOL/cerbere/ -name '*.pyc' -delete

import os
import sys
sys.path.append('/home/mag/Documents/repos/solab/PySOL/cerbere/')

import cerbere

In [88]:
pn = '/media/SOLabNFS2/tmp/sentinel-2_test/S2A_OPER_PRD_MSIL1C_PDMC_20160108T151349_R135_V20160108T082023_20160108T082023.SAFE/GRANULE/S2A_OPER_MSI_L1C_TL_SGS__20160108T115454_A002849_T35HKA_N02.01/IMG_DATA/S2A_OPER_MSI_L1C_TL_SGS__20160108T115454_A002849_T35HKA_B02.jp2'

In [90]:
fd = cerbere.mapper.safemsil1cgranulefile.SAFEMSIL1CGranuleFile(url=pn, native_resolution='10m')

In [93]:
fd.

[]

In [ ]:
pn = '/media/SOLabNFS2/tmp/sentinel-2_test/'
fn = 'S2A_OPER_PRD_MSIL1C_PDMC_20160108T151349_R135_V20160108T082023_20160108T082023.zip'


ext = os.path.splitext(fn)[1].lower()
if ext == '.tiff':
    fileName = [pn + fn]
elif ext == '.zip':
    from zipfile import ZipFile
    import tempfile
    import shutil
    try:
        tmpFldr = tempfile.mkdtemp()  # create tmpdir
        with ZipFile(pn+fn, "r") as zf:
            for name in zf.namelist():
                localFilePath = zf.extract(name, tmpFldr)
    #             print localFilePath
        jpgs = [s for s in zf.namelist() if ".tiff" in s]
        zf.close()
    except Exception:
        Exception.message

    # Construct filename to process
    if len(tiffs) == 2:
        fileName = [tmpFldr + '/' + tiffs[0], tmpFldr + '/' + tiffs[1]]
    else:
        fileName = tmpFldr + '/' + tiffs[0]

In [ ]:
sarmp = sarmapper(fileName)
sarim = SARImage(sarmp)
spacing_m = int0(800)
spacing = int0(sarim.meters2pixels(spacing_m))
# ssr = sarim.get_data('roughness', spacing=spacing)
sigma0 = sarim.get_data('sigma0', spacing=spacing)

In [ ]:
lons = sarim.get_data('lon', spacing=spacing)

lons.min(), lons.max()

In [ ]:
plt.close('all')
plt.imshow(10*log10(fliplr(sigma0)), interpolation='nearest', vmin=-27, vmax=-3)
plt.gray()

In [ ]:
# shutil.rmtree(tmpFldr)  # delete directory
# !rm -rf /tmp/*
# %ls /tmp

### NonInteractive Plot

In [ ]:
from scipy.stats import scoreatpercentile
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.cm as cm

# %matplotlib qt
# %matplotlib inline
# import mpld3
# mpld3.enable_notebook()

plt.close('all')
vmin = None
vmax = None

spacing_ra = ceil(sarim.get_info('number_of_samples')/600.)
spacing_ra_m = sarim.pixels2meters(spacing_ra)[1]
spacing = int0(round(sarim.meters2pixels(spacing_ra_m)))
spacing_m = sarim.pixels2meters(spacing)
sigma0 = sarim.get_data('sigma0', spacing=spacing)
sigma0 = 10*log10(sigma0)
lon = sarim.get_data('lon', midrange=True, midazimuth=True)[0, 0]
lat = sarim.get_data('lat', midrange=True, midazimuth=True)[0, 0]
inc = sarim.get_data('incidence', midrange=True, midazimuth=True)[0, 0]
heading = sarim.get_info('platform_heading')
north = 90 + heading # north dir in image
im_num = sarim.get_info('image_number')
dist = (array(sigma0.shape)-1)*spacing_m/1000.
if vmin is None:
    vmin = scoreatpercentile(sigma0[25:-25, 25:-25], 0.1)
if vmax is None:
    vmax = scoreatpercentile(sigma0[25:-25, 25:-25], 99.9)
if sarim.get_info('pass') == 'Descending':
    sigma0 = sigma0[::-1, ::-1]
    north = north+180

# Make figure
dpi = 100
imsize = array(sigma0.shape[::-1])
margin = array(((900-imsize[0])/2, 60))
figsize = array((900, imsize[1]+2*margin[1]))
imsizeN = imsize.astype('float')/figsize
marginN = margin.astype('float')/figsize

fig = plt.figure(figsize=figsize.astype('float')/dpi, dpi=dpi)
imax = fig.add_axes([marginN[0], marginN[1], imsizeN[0], imsizeN[1]])
imax.set_xlim(0, dist[1])
imax.set_ylim(0, dist[0])
plt.imshow(sigma0, origin='lower', interpolation='nearest', vmin=vmin,
            vmax=vmax, cmap=cm.get_cmap('Greys_r'),
            extent=[0, dist[1], 0, dist[0]], aspect='auto')
imax.set_xlabel('range distance [km]')
imax.set_ylabel('azimuth distance [km]')
tit = '#%03i / lon=%.2f / lat=%.2f / inc=%.2f / pass=%s' % (im_num, lon, lat, inc, sarim.get_info('pass'))
imax.set_title(tit)

# Add colorbar
cbax = fig.add_axes([1-0.75*marginN[0], .25, 20./figsize[0], .50])
plt.colorbar(label='sea surface roughness', cax=cbax, format='%i')
meanstr = r'$\sigma$=%.f' % sigma0.mean()
cbax.text(0.5, -0.025, meanstr, ha='center', va='top')

# Add north
cpsizeN = (margin[0]-margin[1])/figsize.astype('float')
cpax = fig.add_axes([1-0.75*marginN[0], 0.15-cpsizeN[1]/2, cpsizeN[0], cpsizeN[1]])
plt.arrow(.5, .5, .5*cos(north*pi/180),
          .5*sin(north*pi/180), facecolor='black',
          width=0.07, length_includes_head=True,
          head_width=0.2, head_length=0.3)
plt.annotate('North', (.5, .4), ha='center', va='top')
cpax.set_axis_off()

In [ ]:
print spacing, spacing_m, sigma0.shape,

In [ ]:
plt.close('all')
plt.imshow(sigma0)

In [ ]:
from scipy.stats import scoreatpercentile
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.cm as cm

vmin = None
vmax = 7
cmap='Greys_r'

spacing_ra = ceil(sarim.get_info('number_of_samples')/600.)
spacing_ra_m = sarim.pixels2meters(spacing_ra)[1]
spacing = int0(round(sarim.meters2pixels(spacing_ra_m)))
spacing_m = sarim.pixels2meters(spacing)
ssr = sarim.get_data('roughness', spacing=spacing)
lon = sarim.get_data('lon', midrange=True, midazimuth=True)[0, 0]
lat = sarim.get_data('lat', midrange=True, midazimuth=True)[0, 0]
inc = sarim.get_data('incidence', midrange=True, midazimuth=True)[0, 0]
heading = sarim.get_info('platform_heading')
north = 90 + heading # north dir in image
im_num = sarim.get_info('image_number')
dist = (array(ssr.shape)-1)*spacing_m/1000.
if vmin is None:
    vmin = scoreatpercentile(ssr[25:-25, 25:-25], 0.1)
if vmax is None:
    vmax = scoreatpercentile(ssr[25:-25, 25:-25], 99.9)
if sarim.get_info('pass') == 'Descending':
    ssr = ssr[::-1, ::-1]
    north = north+180

# Make figure
dpi = 100
imsize = array(ssr.shape[::-1])
margin = array(((900-imsize[0])/2, 60))
figsize = array((900, imsize[1]+2*margin[1]))
imsizeN = imsize.astype('float')/figsize
marginN = margin.astype('float')/figsize

fig = plt.figure(figsize=figsize.astype('float')/dpi, dpi=dpi)
imax = fig.add_axes([marginN[0], marginN[1], imsizeN[0], imsizeN[1]])
imax.set_xlim(0, dist[1])
imax.set_ylim(0, dist[0])
plt.imshow(ssr, origin='lower', interpolation='nearest', vmin=vmin,
            vmax=vmax,
            extent=[0, dist[1], 0, dist[0]], aspect='auto')
plt.set_cmap(cmap) 
imax.set_xlabel('range distance [km]')
imax.set_ylabel('azimuth distance [km]')
tit = '#%03i / lon=%.2f / lat=%.2f / inc=%.2f / pass=%s' % (im_num, lon, lat, inc, sarim.get_info('pass'))
imax.set_title(tit)

# Add colorbar
cbax = fig.add_axes([1-0.75*marginN[0], .25, 20./figsize[0], .50])
plt.colorbar(label='sea surface roughness', cax=cbax, format='%.1f')
meanstr = r'$\rho$=%.1f' % ssr.mean()
cbax.text(0.5, -0.025, meanstr, ha='center', va='top')

# Add north
cpsizeN = (margin[0]-margin[1])/figsize.astype('float')
cpax = fig.add_axes([1-0.75*marginN[0], 0.15-cpsizeN[1]/2, cpsizeN[0], cpsizeN[1]])
plt.arrow(.5, .5, .5*cos(north*pi/180),
          .5*sin(north*pi/180), facecolor='black',
          width=0.07, length_includes_head=True,
          head_width=0.2, head_length=0.3)
plt.annotate('North', (.5, .4), ha='center', va='top')
cpax.set_axis_off()

In [ ]:
sarim.get_datanames()

In [ ]:
def extractZip2memory(input_zip):
    input_zip=ZipFile(input_zip, 'r')
    return {name: StringIO(input_zip.read(name)) for name in input_zip.namelist()[:-1]}
zf.namelist()

### Interactive Plot - doesn't work

In [ ]:
dpi = 300
figsize = array([3, 3*double(ssr.shape[0])/double(ssr.shape[1])])*dpi
imsize = array(ssr.shape[::-1])
margin = array(((figsize[0]-imsize[0])/2, 60))
imsizeN = imsize.astype('float')/figsize
marginN = margin.astype('float')/figsize

print figsize, imsizeN, imsize, marginN, margin

dpi = 300
imsize = array(ssr.shape[::-1])
margin = array(((900-imsize[0])/2, 60))
figsize = array((900, imsize[1]+2*margin[1]))
imsizeN = imsize.astype('float')/figsize
marginN = margin.astype('float')/figsize

print figsize, imsizeN, imsize, marginN, margin

imsize = array((1200,1200*ssr.shape[1]/ssr.shape[0]))
margin = array(((900-imsize[0])/2, 60))
figsize = array((900, imsize[1]+2*margin[1]))
imsizeN = imsize.astype('float')/figsize
marginN = margin.astype('float')/figsize
fig = plt.figure(figsize=figsize.astype('float')/dpi, dpi=dpi)

margin = array(((900-imsize[0])/2, 60))
figsize = array((900, imsize[1]+2*margin[1]))

In [ ]:
from scipy.stats import scoreatpercentile
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.cm as cm

vmin = None
vmax = None

# spacing_ra = ceil(sarim.get_info('number_of_samples')/2400.)
# spacing_ra_m = sarim.pixels2meters(spacing_ra)[1]
# spacing = round(sarim.meters2pixels(spacing_ra_m))
# spacing_m = sarim.pixels2meters(spacing)
ssr = sarim.get_data('roughness', spacing=spacing)
lon = sarim.get_data('lon', midrange=True, midazimuth=True)[0, 0]
lat = sarim.get_data('lat', midrange=True, midazimuth=True)[0, 0]
inc = sarim.get_data('incidence', midrange=True, midazimuth=True)[0, 0]
heading = sarim.get_info('platform_heading')
north = 90 + heading # north dir in image
im_num = sarim.get_info('image_number')
dist = (array(ssr.shape)-1)*spacing_m/1000.
if vmin is None:
    vmin = scoreatpercentile(ssr[25:-25, 25:-25], 0.1)
if vmax is None:
    vmax = scoreatpercentile(ssr[25:-25, 25:-25], 99.9)
if sarim.get_info('pass') == 'Descending':
    ssr = ssr[::-1, ::-1]
    north = north+180

def ntrctv_imshow(vmin=None, vmax=None, cmap='Greys_r', crop='[:,:]'):
    # Make figure
    dpi = 300
    imsize = array(ssr.shape[::-1])
    margin = array(((900-imsize[0])/2, 60))
    figsize = array((900, imsize[1]+2*margin[1]))
    imsizeN = imsize.astype('float')/figsize
    marginN = margin.astype('float')/figsize


#     figsize = array([16, 16*double(ssr.shape[0])/double(ssr.shape[1])])*dpi
#     imsize = array(ssr.shape[::-1])
#     margin = array(((figsize[0]-imsize[0])/2, 60))
#     imsizeN = imsize.astype('float')/figsize
#     marginN = margin.astype('float')/figsize

    fig = plt.figure(figsize=figsize.astype('float')/dpi, dpi=dpi)
    imax = fig.add_axes([marginN[0], marginN[1], imsizeN[0], imsizeN[1]])
    imax.set_xlim(0, dist[1])
    imax.set_ylim(0, dist[0])
    plt.imshow(eval("ssr"+str(crop)), origin='lower', interpolation='nearest', vmin=vmin,
                vmax=vmax,
                extent=[0, dist[1], 0, dist[0]], aspect='auto')
    plt.set_cmap(cmap) 
    imax.set_xlabel('range distance [km]')
    imax.set_ylabel('azimuth distance [km]')
    tit = '#%03i / lon=%.2f / lat=%.2f / inc=%.2f / pass=%s' % (im_num, lon, lat, inc, sarim.get_info('pass'))
    imax.set_title(tit)

    # Add colorbar
    cbax = fig.add_axes([1-0.75*marginN[0], .25, 20./figsize[0], .50])
    plt.colorbar(label='sea surface roughness', cax=cbax, format='%.1f')
    meanstr = r'$\rho$=%.1f' % ssr.mean()
    cbax.text(0.5, -0.025, meanstr, ha='center', va='top')

    # Add north
    cpsizeN = (margin[0]-margin[1])/figsize.astype('float')
    cpax = fig.add_axes([1-0.75*marginN[0], 0.15-cpsizeN[1]/2, cpsizeN[0], cpsizeN[1]])
    plt.arrow(.5, .5, .5*cos(north*pi/180),
              .5*sin(north*pi/180), facecolor='black',
              width=0.07, length_includes_head=True,
              head_width=0.2, head_length=0.3)
    plt.annotate('North', (.5, .4), ha='center', va='top')
    cpax.set_axis_off()



from IPython.html import widgets

ntrctv = widgets.interact(ntrctv_imshow, \
                 vmin=widgets.FloatSliderWidget(min=0, max=5, value=vmin, step=0.2), \
                 vmax=widgets.FloatSliderWidget(min=0, max=5, value=vmax, step=0.2), \
                 cmap = ['Greys_r', 'bone', 'RdBu_r'], crop='[:,:]');